# Object Relational Tutorial

source: https://docs.sqlalchemy.org/en/13/orm/tutorial.html

The SQLAlchemy Object Relational Mapper presents a method of associating:
1. user-defined Python classes with database tables, and
2. instances of those classes (objects) with rows in their corresponding tables.

It includes:
1. a system that transparently synchronizes all changes in state between objects and their related rows, called a [unit of work](https://docs.sqlalchemy.org/en/13/glossary.html#term-unit-of-work), as well as
2. a system for expressing database queries in terms of the user defined classes and their defined relationships between each other.

In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.19'

In [2]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)

type(engine)

sqlalchemy.engine.base.Engine

When using the ORM, we typically don’t use the `Engine` directly once created; instead, it’s used behind the scenes by the ORM as we’ll see shortly.

### Declare a Mapping

When using the ORM, the configurational process:
- starts by describing the database tables we’ll be dealing with, and
- then by defining our own classes which will be mapped to those tables.

In modern SQLAlchemy, these two tasks are usually performed together, using a system known as `Declarative`, which allows us to create classes that include directives to describe the actual database table they will be mapped to.

Classes mapped using the `Declarative` system are defined in terms of a base class which maintains a catalog of classes and tables relative to that base - this is known as the **declarative base class**.

In [3]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [4]:
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return (
            f"<User(name={self.name}, fullname={self.fullname},"
            f" nickname={self.nickname})>"
        )

When our class is constructed, Declarative replaces all the `Column` objects with special Python accessors known as "descriptors"; this is a process known as "instrumentation".

The “instrumented” mapped class will provide us with the means to refer to our table in a SQL context as well as to persist and load the values of columns from the database.

### Create a Schema and an Instance of the Mapped Class

In [5]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [6]:
type(User.__table__)

sqlalchemy.sql.schema.Table

In [7]:
# issue CREATE TABLE statements to the database
# for all tables that don’t yet exist

Base.metadata.create_all(engine)

2020-10-14 08:12:57,996 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-14 08:12:57,998 INFO sqlalchemy.engine.base.Engine ()
2020-10-14 08:12:57,999 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-14 08:12:58,006 INFO sqlalchemy.engine.base.Engine ()
2020-10-14 08:12:58,015 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-10-14 08:12:58,016 INFO sqlalchemy.engine.base.Engine ()
2020-10-14 08:12:58,022 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-10-14 08:12:58,025 INFO sqlalchemy.engine.base.Engine ()
2020-10-14 08:12:58,028 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2020-10-14 08:12:58,033 INFO sqlalchemy.engine.base.Engine ()
2020-10-14 08:12:58,037 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')

ed_user

<User(name=ed, fullname=Ed Jones, nickname=edsnickname)>

In [9]:
str(ed_user.id)

'None'

### Creating a Session

We’re now ready to start talking to the database. The ORM’s “handle” to the database is the `Session`.

In the most general sense, the `Session` establishes all conversations with the database and represents a “holding zone” for all the objects which you’ve loaded or associated with it during its lifespan.

In [10]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [11]:
session = Session()

In [12]:
session.add(ed_user)

At this point, we say that the instance is **pending**; no SQL has yet been issued and the object is not yet represented by a row in the database.

The `Session` will issue the SQL to persist `ed_user` as soon as is needed; this process is known as a **flush**. For example, if we query the database:
- first all pending information will be flushed, and
- immediately thereafter the query will be issued.

In [13]:
query = session.query(User).filter_by(name='ed')

print(type(query))
print()

our_user = query.first()

<class 'sqlalchemy.orm.query.Query'>
2020-10-14 08:12:58,163 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-14 08:12:58,168 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-14 08:12:58,170 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2020-10-14 08:12:58,174 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2020-10-14 08:12:58,179 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


In [14]:
# Note: the Session has identified that
# the row returned is **the same row**
# as one already represented within its internal map of objects,
# so we actually got back the identical instance
# as that which we just added:

# Note: executing the `Query` returned the very same instance
# as the one that was added to the `Session`:
our_user is ed_user

True

[The `Session`] provides the entrypoint to acquire a `Query` object, which sends queries to the database (using the `Session` object’s current database connection).

Doing so populates result rows into objects that are then stored in the `Session`, inside a structure called the "Identity Map" - a data structure that maintains unique copies of each object, where “unique” means “only one object with a particular primary key”.

An Identity Map is a mapping that’s associated with an ORM `Session` object, and maintains a mapping/correspondence between Python objects and their database identities. (Thus, an Identity Map keeps a record of all objects that have been read from the database in a single business transaction.)

In [15]:
# In addition,
# if we look at Ed's `id` attribute, which earlier was `None`,
# it now has a value:
ed_user.id, our_user.id

(1, 1)

In [16]:
# We can add more User objects at once:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy'),
])

# Also, let's change Ed's nickname:
ed_user.nickname = 'eddie'

In [17]:
# The `Session` is paying attention...
# exhibit A:
session.dirty

IdentitySet([<User(name=ed, fullname=Ed Jones, nickname=eddie)>])

In [18]:
# exhibit B:
session.new

IdentitySet([<User(name=wendy, fullname=Wendy Williams, nickname=windy)>, <User(name=mary, fullname=Mary Contrary, nickname=mary)>, <User(name=fred, fullname=Fred Flintstone, nickname=freddy)>])

In [19]:
# issue all remaining changes to the database and commit the transaction,
# which has been in progress throughout
session.commit()

2020-10-14 08:12:58,314 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2020-10-14 08:12:58,316 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2020-10-14 08:12:58,321 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-14 08:12:58,324 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2020-10-14 08:12:58,329 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-14 08:12:58,330 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2020-10-14 08:12:58,331 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-14 08:12:58,333 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2020-10-14 08:12:58,335 INFO sqlalchemy.engine.base.Engine COMMIT


### Rolling back

Since the `Session` works within a transaction, we can roll back changes made too.

In [20]:
# Make 2 changes that we'll revert

ed_user.name = 'Edwardo'

fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)

In [21]:
# Querying the session shows that
# both changes are flushed into the current transaction
session.query(User).filter(
    User.name.in_(['Edwardo', 'fakeuser'])
).all()

2020-10-14 08:24:11,562 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-14 08:24:11,569 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2020-10-14 08:24:11,573 INFO sqlalchemy.engine.base.Engine (1,)
2020-10-14 08:24:11,580 INFO sqlalchemy.engine.base.Engine UPDATE users SET name=? WHERE users.id = ?
2020-10-14 08:24:11,581 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)
2020-10-14 08:24:11,583 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2020-10-14 08:24:11,586 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '12345')
2020-10-14 08:24:11,589 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2020-10-14 08:24:11,591 INFO sqlalchemy.engine.base.Engine ('Ed

[<User(name=Edwardo, fullname=Ed Jones, nickname=eddie)>,
 <User(name=fakeuser, fullname=Invalid, nickname=12345)>]

In [31]:
# Roll back the changes, and verify that has worked as expected.

session.rollback()

print(ed_user.name == 'ed')
print(fake_user in session)

print()
print(
    session.query(User).filter(
        User.name.in_(['ed', 'fakeuser'])
    ).all()
)

2020-10-14 08:28:32,032 INFO sqlalchemy.engine.base.Engine ROLLBACK
2020-10-14 08:28:32,036 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-14 08:28:32,047 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2020-10-14 08:28:32,055 INFO sqlalchemy.engine.base.Engine (1,)
True
False

2020-10-14 08:28:32,073 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2020-10-14 08:28:32,076 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')
[<User(name=ed, fullname=Ed Jones, nickname=eddie)>]
